In [40]:
import numpy as np
import cv2
import os

In [41]:
#Adding all images to one list with classes into another.

dataset_path = 'OCR_Dataset\RecognizerDataset_150_210'
images = []
class_ids = []
dirs = os.listdir(dataset_path)
no_of_classes = len(dirs)
class_id_counter = 0
for class_id in dirs:
    imageList = os.listdir(os.path.join(dataset_path, str(class_id)))
    for image_name in imageList:
        curImg = cv2.imread(os.path.join(dataset_path, str(class_id), str(image_name)))
        curImg = cv2.resize(curImg, (32,32))
        images.append(curImg)
        class_ids.append(class_id_counter)
    print(class_id , class_id_counter, end= " ")
    class_id_counter+=1

a 0 a(3) 1 a(4) 2 a(5) 3 aaa 4 ba 5 ba(2) 6 baa 7 be 8 bha 9 bhaa 10 bhe 11 bhi 12 bhii 13 bho 14 bhu 15 bhuu 16 bi 17 bii 18 bo 19 bo(2) 20 bu 21 buu 22 ca 23 caa 24 ce 25 cha 26 chaa 27 che 28 chi 29 chii 30 cho 31 chu 32 chuu 33 ci 34 cii 35 co 36 cu 37 cuu 38 da(2) 39 daa 40 daa(2) 41 daaa 42 daaaa 43 dae 44 dai 45 daii 46 dao 47 dau 48 dauu 49 de 50 dha 51 dhaa 52 dhaaa 53 dhaaaa 54 dhae 55 dhai 56 dhaii 57 dhao 58 dhau 59 dhauu 60 dhi 61 dhii 62 dho 63 dhu 64 dhue 65 dhuu 66 di 67 dii 68 do 69 du 70 duu 71 e 72 ee 73 ga 74 gaa 75 ge 76 gha 77 ghaa 78 ghe 79 ghi 80 ghii 81 gho 82 ghu 83 ghuu 84 gi 85 gii 86 go 87 gu 88 guu 89 ha 90 haa 91 he 92 hi 93 hii 94 ho 95 hu 96 huu 97 i 98 ja 99 ja(2) 100 ja(3) 101 ja(4) 102 jaa 103 je 104 jha 105 jhaa 106 jhe 107 jhi 108 jhii 109 jho 110 jhu 111 jhuu 112 ji 113 jii 114 jo 115 ju 116 juu 117 ka 118 kaa 119 ke 120 kha 121 kha(2) 122 khaa 123 khaa(2) 124 khe 125 khe(2) 126 khi 127 khii 128 khii(2) 129 kho 130 kho(2) 131 khu 132 khu(2) 133 kh

In [42]:
images = np.array(images)
class_ids = np.array(class_ids)
print(images.shape)
print(class_ids.shape)

(60048, 32, 32, 3)
(60048,)


In [43]:
from sklearn.model_selection import train_test_split

In [44]:
#Train test split
test_ratio = 0.2
validation_ratio = 0.2
X_train, X_test, y_train, y_test =train_test_split(images, class_ids, test_size=test_ratio)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_ratio)
print("Training : " + str(X_train.shape))
print("Testing : " + str(X_test.shape))
print("Validation : " + str(X_validation.shape))


Training : (38430, 32, 32, 3)
Testing : (12010, 32, 32, 3)
Validation : (9608, 32, 32, 3)


In [45]:
def dilate(img):
    kernel = np.ones((3, 3), np.uint8)
    return cv2.dilate(img, kernel, iterations=1)


X_train = np.array(list(map(dilate, X_train)))
X_test = np.array(list(map(dilate, X_test)))
X_validation = np.array(list(map(dilate, X_validation)))


In [46]:
#Preprocessing
def pre_process(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255
    return img


    
""" img = pre_process(X_train[45])
img = cv2.resize(img, (300, 300))
cv2.imshow("PreProcessed", img)
cv2.waitKey(0) """

X_train = np.array(list(map(pre_process, X_train)))
X_test = np.array(list(map(pre_process, X_test)))
X_validation = np.array(list(map(pre_process, X_validation)))

print(X_train[45])
print(X_test[45].shape)
print(X_validation[45].shape)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
(32, 32)
(32, 32)


In [47]:
#Adding depth of 1 for NN
print(X_train.shape)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_validation = X_validation.reshape(X_validation.shape[0], X_validation.shape[1], X_validation.shape[2], 1)
print(X_train[45].shape)


(38430, 32, 32)
(32, 32, 1)


In [48]:
from keras.preprocessing.image import ImageDataGenerator

In [49]:
# Data Augmentation
'''
dataGen = ImageDataGenerator(width_shift_range=0.05, height_shift_range=0.05, zoom_range=0.1, shear_range=0.1,rotation_range=5)
dataGen.fit(X_train)'''


'\ndataGen = ImageDataGenerator(width_shift_range=0.05, height_shift_range=0.05, zoom_range=0.1, shear_range=0.1,rotation_range=5)\ndataGen.fit(X_train)'

In [50]:
from keras.utils.np_utils import to_categorical

In [51]:
#Converting to one-hot encoding
y_train = to_categorical(y_train, no_of_classes)
y_test = to_categorical(y_test, no_of_classes)
y_validation = to_categorical(y_validation, no_of_classes)

print(y_train[45])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [52]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt

In [53]:
#Model
def myModel():
    no_of_filters = 60
    size_of_filter1 = (5,5)
    size_of_filter2 = (3,3)
    size_of_pool = (2,2)
    no_of_nodes = 500

    model = Sequential()
    model.add((Conv2D(no_of_filters, size_of_filter1, input_shape = (32,32,1), activation='relu')))
    model.add((Conv2D(no_of_filters, size_of_filter1, activation='relu')))
    model.add(MaxPooling2D(pool_size=size_of_pool))
    model.add((Conv2D(no_of_filters//2, size_of_filter2, activation='relu')))
    model.add((Conv2D(no_of_filters//2, size_of_filter2, activation='relu')))
    model.add(MaxPooling2D(pool_size=size_of_pool))

    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(no_of_nodes, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(no_of_classes, activation='softmax'))
    model.compile('adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
    return model

model = myModel()
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 28, 28, 60)        1560      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 60)        90060     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 60)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 10, 10, 30)        16230     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 8, 8, 30)          8130      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 30)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 4, 4, 30)         

In [54]:

epochs_val = 25
batch_size_val = 50
steps_per_epoch_val = 38000//batch_size_val
validation_steps_val = 2000

#history = model.fit(X_train, y_train, batch_size=batch_size_val, epochs=epochs_val, steps_per_epoch=steps_per_epoch_val, validation_steps=validation_steps_val, validation_data=(X_validation, y_validation), shuffle=1)

# Without augmentation
history = model.fit(X_train, y_train, epochs=epochs_val, validation_data=(X_validation, y_validation), shuffle=1)

# With augmentation
#history = model.fit_generator(dataGen.flow(X_train, y_train, batch_size=batch_size_val),  steps_per_epoch=steps_per_epoch_val, epochs=epochs_val, validation_data=(X_validation, y_validation), shuffle=1)

plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'])
plt.title('Loss')
plt.xlabel('epoch')

plt.figure(2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training', 'validation'])
plt.title('Accuracy')
plt.xlabel('epoch')
plt.show()

score = model.evaluate(X_test, y_test, verbose=0)
print('Test Score =', score[0])
print('Test Accuracy =', score[1])

Train on 38430 samples, validate on 9608 samples
Epoch 1/25
38430/38430 [==============================] - 112s 3ms/step - loss: 5.6611 - accuracy: 0.0033 - val_loss: 5.6614 - val_accuracy: 0.0026
Epoch 2/25
38430/38430 [==============================] - 112s 3ms/step - loss: 5.5740 - accuracy: 0.0104 - val_loss: 5.4441 - val_accuracy: 0.0220
Epoch 3/25
38430/38430 [==============================] - 108s 3ms/step - loss: 5.4037 - accuracy: 0.0235 - val_loss: 5.3249 - val_accuracy: 0.0348
Epoch 4/25
38430/38430 [==============================] - 115s 3ms/step - loss: 5.3226 - accuracy: 0.0317 - val_loss: 5.2828 - val_accuracy: 0.0401
Epoch 5/25
38430/38430 [==============================] - 116s 3ms/step - loss: 5.2754 - accuracy: 0.0376 - val_loss: 5.2384 - val_accuracy: 0.0490
Epoch 6/25
38430/38430 [==============================] - 105s 3ms/step - loss: 5.2389 - accuracy: 0.0434 - val_loss: 5.2063 - val_accuracy: 0.0584
Epoch 7/25
38430/38430 [==============================] - 106s 

KeyboardInterrupt: 

In [ ]:
import pickle

In [ ]:
pickle_out = open("ocr_model_trained_with_dilation", "wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [ ]:
Capstone_Handwritten_Letters_Path = "C:\MyStuff\PES\Capstone_Handwritten_Letters"
test_images = []
test_class_ids = []
dir_to_num = {}
num_to_name = {}
class_id_counter = 0

for class_id in dirs:
        dir_to_num[class_id] = class_id_counter
        num_to_name[class_id_counter] = class_id
        class_id_counter+=1

imageList = os.listdir(Capstone_Handwritten_Letters_Path)
for image_name in imageList:
        curImg = cv2.imread(os.path.join(Capstone_Handwritten_Letters_Path, str(image_name)))
        curImg = cv2.resize(curImg, (32,32))
        test_images.append(curImg)
        test_class_ids.append(dir_to_num[image_name[:-4]])
        
test_images = np.array(test_images)
test_class_ids = np.array(test_class_ids)

test_images = np.array(list(map(pre_process, test_images)))

test_images = test_images.reshape(test_images.shape[0], test_images.shape[1], test_images.shape[2], 1)
print(test_images.shape)
test_class_ids = to_categorical(test_class_ids, no_of_classes)

score = model.evaluate(test_images, test_class_ids, verbose=0)
print('Test Score =', score[0])
print('Test Accuracy =', score[1]) 

'''
predictions = model.predict(test_images)
predictions = np.array(predictions)
print(predictions.shape)
print(len(predictions[0]))

test_preds = []
for pred in predictions:
        max_val = -1
        index_val = -1
        for idx, val in enumerate(pred):
                if val > max_val:
                        max_val = val
                        index_val = idx

        test_preds.append(num_to_name[index_val])
'''
predictions = model.predict_classes(test_images)
for pred in predictions:
        print(num_to_name[pred])
#print(test_preds)


(45, 32, 32, 1)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Test Score = 3.389374007119073
Test Accuracy = 0.2888889014720917
phuu
aaa
haa
bhe
thuu
buu
thuu
che
chu
taii
a
muu
le
ro
thai
dhi
raa
ghu
pi
gaa
haa
dauu
ro
ke
kho
aaa
thaii
ro
nnu
nnii
o
paa
pe
phuu
re
re
khii
shai
she
sii
so
jhi
te
thauu
ya
